In [ ]:
'''
Tabulate SQLite example using a database derived from Fukushima data on data.gov.

The converter created four types of records:
1. Source - Paths to source and data dictionary files
2. Experiments - One record per flight
3. Observations - Each observation taken during each of the three flights
4. Units - One record for each measurement to provide efficient representation of
           units and a description

plus relationship records between experiments and observations.

This notebook demonstrates using Sina queries to display data in tables.  One
table is created for each record type and a final one for the relationships
between experiment and observation records.

Source: https://catalog.data.gov/dataset/\
    us-doe-nnsa-response-to-2011-fukushima-incident-at-sea-aerial-data
'''
import json
import ipywidgets as widgets
import IPython.display as ipyd
import sys
import tabulate

# TODO: Change the path below to your sina installation if the following doesn't work for you.
sys.path.append('/collab/usr/gapps/wf/releases/venv-user/lib/python2.7/site-packages/sina')

import sina.datastores.sql as sds
import sina.datastores.sql_schema as sdss

# Create the database access factory.
# (NOTE:  The following data is currently only available on the CZ.)
fact = sds.DAOFactory('/collab/usr/gapps/wf/examples/data/fukushima/fukushima.sqlite')
#fact = sds.DAOFactory('/g/g21/dahlgren/siboka/examples/fukushima/files/fukushima.sqlite')


In [ ]:
# Extract source record(s) from the database
source = fact.createRecordDAO().get_all_of_type("source")

# Build a list of table entries, one experiment per table row
data = [
    ['Source Id', 'File URI', 'File Type', 'File Tag']
]

for src in source:
    rec = src.raw
    sid = rec['id'].encode('utf-8')

    for fl in rec['files']:
        uri = fl['uri'].encode('utf-8')
        ftp = fl['mimetype'].encode('utf-8')
        ftg = fl['tags'][0].encode('utf-8')
        data.append([sid, uri, ftp, ftg])

# Display the data in an HTML table
tbl = tabulate.tabulate(data, tablefmt='html')
ipyd.display(ipyd.HTML(tbl))

In [ ]:
# Extract the units records from the database
lunits = fact.createRecordDAO().get_all_of_type("units")

# Prepare to save off the units for later use
dunits = {}

# Build a list of table entries, one quality control entry per row
data = [
    ['Measurement', 'Units', 'Description']
]

lkeys = ['measure', 'description']
for un in lunits:
    rec = un.raw
    units = rec['data'][0]['value'].encode('utf-8')
    desc = rec['data'][1]['value'].encode('utf-8')
    data.append([rec['id'], units, desc])
    
    dunits[rec['id']] = units

# Display the data in an HTML table
tbl = tabulate.tabulate(data, tablefmt='html')
ipyd.display(ipyd.HTML(tbl))

In [ ]:
# Extract experiment record(s) from the database
exps = fact.createRecordDAO().get_all_of_type("exp")

# Build a list of table entries, one experiment per table row
data = [
    ['Experiment Id', 'Data Source']
]

eids = []  # Save off the experiment ids for the relationship queries
for exp in exps:
    rec = exp.raw
    eid = rec['id'].encode('utf-8')
    eids.append(eid)

    fl = rec['files'][0]['uri'].encode('utf-8')
    data.append([eid, fl])

# Display the data in an HTML table
tbl = tabulate.tabulate(data, tablefmt='html')
ipyd.display(ipyd.HTML(tbl))

In [ ]:
# Extract the observation records from the database
lobs = fact.createRecordDAO().get_all_of_type("obs")

# Build a list of table entries, one observation per table row
data = [
    ['Observation Id', 'Date', 'Time', 'Latitude', 'Longitude',
     'GPS Altitude','Sea Level Altitude', 'Live Time', 'Gross Counts']
]
lkeys = ['date', 'time', 'latitude', 'longitude', 'alt_hae', 'agl', 'live','gcnorm']

# .. Add units to the header
for i, k in enumerate(lkeys):
    if k in dunits.keys():
        data[0][i+1] = '%s (%s)' % (data[0][i+1], dunits[k])

# .. Add observations to the output
for obs in lobs:
    rec = obs.raw
    vals = [rec['id'].encode('utf-8')]
    vals.extend(['' for _u in lkeys])
    for row in obs.raw['data']:
        nm = row['name']
        lkys = row.keys()
        if nm in lkeys:
            ind = lkeys.index(nm)
            val = row['value']
            if isinstance(val, unicode):
                val = val.encode('utf-8')
            vals[ind+1] = val
    data.append(vals)

# Display the data in an HTML table
tbl = tabulate.tabulate(data, tablefmt='html')
ipyd.display(ipyd.HTML(tbl))

In [ ]:
# Extract relationship records from the database, for each experiment
dao = fact.createRelationshipDAO()

# Build a list of table entries, one quality control entry per row
data = [
    ['Experiment Id', 'Relationship', 'Observation Id']
]

for eid in eids:
    rels = dao.get(subject_id=eid)
    for rel in rels:
        pred = rel.predicate.encode('utf-8')
        oid = rel.object_id.encode('utf-8')
        data.append([eid, pred, oid])

# Display the data in an HTML table
tbl = tabulate.tabulate(data, tablefmt='html')
ipyd.display(ipyd.HTML(tbl))